In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback
import copy 

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./bm1/bohb/case/")
target_dirs = os.listdir(path)

In [3]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_info = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
#         print(dict_r)
        
        if dict_r[3] is None:
            continue
        else:
                #         exec_time = dict_r[2]['finished'] - dict_r[2]['started']
            exec_time = dict_r[3]['info']['elapsed time']
    #         opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']
            opt_time = 0

            if cum_first is True:
                cum_exec_time_last_value = 0.0
                cum_opt_time_last_value = 0.0
                cum_first = False

            if cum_exec_time_last_value <= 86400:
                exec_times.append(exec_time)
                opt_times.append(opt_time)
                cum_exec_time.append((cum_exec_time_last_value + exec_time))
                cum_opt_time.append((cum_opt_time_last_value + opt_time))
                epochs.append(dict_r[1])
                model_info.append(dict_r[0])
                errors.append(dict_r[3]['loss'])
                accuracies.append(dict_r[3]['info']['test accuracy'])

                cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
                cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]

                dict_result = dict([("cum_exec_time", cum_exec_time),
                            ("exec_time", exec_times),
                            ("opt_time", opt_times),
                            ("model_info", model_info),
                            ("cum_opt_time", cum_opt_time),
                            ("error", errors),
                            ("accuracy", accuracies),
                            ("epoch", epochs)])
            else:
                continue

        
    return dict_result

In [4]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [5]:
def getTrialHyperparams(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[1]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [6]:
def getTrialPerformance(result):
    result_dict = dict()
    index = 0
    for r in result:
        dict_r = json.loads(r)
        train_epoch = dict_r[1]
        
        train_result = copy.copy(dict_r[3])
        train_result['epoch'] = train_epoch
        train_result['key'] = str(dict_r[0])
        #key = "{}({})".format(train_result['key'], train_result['epoch'])
        result_dict[str(index)] = train_result
        index += 1
        #print("{}:{}".format(str(dict_r[0]), train_epoch))
        
    return result_dict

In [7]:
def addModelInfo(trial_result, trial_config):
    model_index_list = []
    for i in range(len(trial_result['model_info'])):
        model_idx = str(trial_result['model_info'][i])
        model_info = trial_config[model_idx]
        target_index = -2
        if 'lookup_index' in model_info:
            target_index = model_info['lookup_index']
        elif trial_result['error'][i] == 1:
            target_index = 0
        model_index_list.append(target_index)
    trial_result['model_index'] = model_index_list
    del trial_result['model_info']

In [8]:
bohb_results = dict()
bohb_configs = dict()
train_results = dict()

for d in target_dirs:
# #     print(d)
#     if d != 'iter10_result':
#         continue
#     else:
#         trial_num = '0'
# #     trial_num = d[5:]
    trial_num = d
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    train_results[trial_num] = getTrialPerformance(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
    bohb_configs[trial_num]=getTrialHyperparams(config)
    
    config_dict=getTrialConfig(config)
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict
    

with open('sample.json', 'w') as outfile:
    json.dump(bohb_results, outfile)

In [9]:
train_results['bad']['0']


{'epoch': 1.0,
 'info': {'elapsed time': 7.243467569351196,
  'number of parameters': 395917,
  'test accuracy': 0.5921,
  'train accuracy': 0.584981818186153,
  'validation accuracy': 0.613},
 'key': '[0, 0, 0]',
 'loss': 0.387}

In [10]:
bohb_configs['bad']['[0, 0, 0]']

{'dropout_rate': 0.7428999559820086,
 'lr': 0.04165651334126147,
 'num_conv_layers': 1,
 'num_fc_units': 73,
 'num_filters_1': 32,
 'optimizer': 'Adam'}

In [12]:
import copy 

merged = {}
trials = bohb_configs.keys()
for t in trials:
    ml = []
    for k in list(train_results[t].keys()):
        key = train_results[t][k]['key']
        m = copy.copy(bohb_configs[t][key])
        m['condition'] = key
        m['train_epoch'] = train_results[t][k]['epoch']
        m['loss'] = train_results[t][k]['loss']        
        ml.append(m)
    merged[t] = ml


In [13]:
merged['bad'][1]

{'condition': '[0, 0, 1]',
 'dropout_rate': 0.09910944192017616,
 'loss': 0.3274,
 'lr': 0.0010348164918088211,
 'num_conv_layers': 1,
 'num_fc_units': 12,
 'num_filters_1': 4,
 'optimizer': 'SGD',
 'sgd_momentum': 0.7924646641779861,
 'train_epoch': 1.0}

In [14]:
bad_results= pd.DataFrame.from_dict(merged['bad'])
bad_results.to_csv('bad_result.csv', index=False)

In [15]:
good_results= pd.DataFrame.from_dict(merged['good'])
good_results.to_csv('good_result.csv', index=False)

In [16]:
bad_results

,condition,dropout_rate,loss,lr,num_conv_layers,num_fc_units,num_filters_1,num_filters_2,num_filters_3,optimizer,sgd_momentum,train_epoch
0,"[0, 0, 0]",0.742900,0.3870,0.041657,1,73,32,NaN,NaN,Adam,NaN,1.0
1,"[0, 0, 1]",0.099109,0.3274,0.001035,1,12,4,NaN,NaN,SGD,0.792465,1.0
2,"[0, 0, 2]",0.623012,0.5954,0.017155,1,8,14,NaN,NaN,Adam,NaN,1.0
3,"[0, 0, 3]",0.142576,0.8940,0.000936,2,22,23,44.0,NaN,SGD,0.945122,1.0
4,"[0, 0, 4]",0.737900,0.8940,0.000003,2,176,8,6.0,NaN,SGD,0.902710,1.0
5,"[0, 0, 5]",0.834977,0.8874,0.004395,3,44,6,16.0,4.0,Adam,NaN,1.0
6,"[0, 0, 6]",0.684580,0.8940,0.000068,3,37,6,58.0,4.0,Adam,NaN,1.0
7,"[0, 0, 7]",0.267093,0.9034,0.000811,2,33,5,34.0,NaN,SGD,0.484609,1.0
8,"[0, 0, 8]",0.852169,0.9022,0.002471,1,9,9,NaN,NaN,SGD,0.841329,1.0
9,"[0, 0, 0]",0.742900,0.8940,0.041657,1,73,32,NaN,NaN,Adam,NaN,3.0
